# Задание 1. Классификация средствами PyTorch

Используя свой датасет для классификации из курса дисциплины "Машинное обучение" описать и обучить модель для классификации с помощью PyTorch. Выбрать функцию потерь. Использовать объекты TensorDataset и DataLoader.

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import NearMiss

In [19]:
data = pd.read_csv('../data/csgo_task.csv')

In [20]:
data.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0


## Обработка

- time_left - длительность раунда
- ct_score - количество очков у команды контр-террористов
- t_score - количество очков у террористов
- map - карта
- bomb_planted - установлена ли бомба
- ct_health и t_health - суммарное здоровье контр-террористов и террористов соответственно
- ct_armor и t_armor - суммарная броня контр-террористов и террористов соответственно
- ct_money и t_money - суммарное количество денег контр-террористов и террористов соответственно
- ct_helmets и t_helmets - суммарное количество шлемов контр-террористов и террористов соответственно
- ct_defuse_kits - количество наборов обезвреживания бомбы у контр-террористо
- ct_players_alive и t_players_alive - количество живых контр-террористов и террористов соответственно

In [4]:
data.drop(['map'], axis=1, inplace=True)

In [5]:
data.dropna(inplace=True)

In [6]:
data.isna().sum()

time_left           0
ct_score            0
t_score             0
bomb_planted        0
ct_health           0
t_health            0
ct_armor            0
t_armor             0
ct_money            0
t_money             0
ct_helmets          0
t_helmets           0
ct_defuse_kits      0
ct_players_alive    0
t_players_alive     0
dtype: int64

In [7]:
data = data.drop_duplicates().reset_index(drop=True)

In [8]:
columns_to_int = [
    "ct_score",
    "t_score",
    "ct_health",
    "t_health",
    "ct_armor",
    "t_armor",
    "ct_money",
    "t_money",
    "ct_helmets",
    "t_helmets",
    "ct_defuse_kits",
    "ct_players_alive",
    "t_players_alive",
    "bomb_planted",
]

for column in columns_to_int:
    data[column] = data[column].astype(int)

## Создание модели

In [9]:
y, X = data['bomb_planted'].values, data.drop(columns=['bomb_planted']).values
y = y.reshape(-1, 1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((95189, 14), (16799, 14), (95189, 1), (16799, 1))

In [ ]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
nm = NearMiss()
X_train, y_train = nm.fit_resample(X_train, y_train.ravel())

In [11]:
train_ds = TensorDataset(torch.from_numpy(X_train).type(torch.float32), torch.from_numpy(y_train).type(torch.float32))
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)

In [12]:
test_ds = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_dl = DataLoader(test_ds, batch_size=256, shuffle=True)

In [13]:
class MyClassificationModel(nn.Module):
    def __init__(self):
        super(MyClassificationModel, self).__init__()
        self.first_linear = nn.Linear(14, 120)
        self.first_relu = nn.ReLU()
        self.second_linear = nn.Linear(120, 240)
        self.second_relu = nn.ReLU()
        self.third_linear = nn.Linear(240, 60)
        self.third_relu = nn.ReLU()
        self.fourth_linear = nn.Linear(60, 20)
        self.fourth_relu = nn.ReLU()
        self.fifth_linear = nn.Linear(20, 1)

    def forward(self, x):
        y = self.first_linear(x)
        y = self.first_relu(y)
        y = self.second_linear(y)
        y = self.second_relu(y)
        y = self.third_linear(y)
        y = self.third_relu(y)
        y = self.fourth_linear(y)
        y = self.fourth_relu(y)
        y = self.fifth_linear(y)
        return y

In [14]:
model = MyClassificationModel()

In [15]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

In [16]:
epochs = 50
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        outputs = model(x_b)
        loss_value = loss(outputs, y_b)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 0.9677443504333496
Эпоха 2, Значение функции потерь: 0.25871559977531433
Эпоха 3, Значение функции потерь: 0.9090948104858398
Эпоха 4, Значение функции потерь: 0.313220351934433
Эпоха 5, Значение функции потерь: 0.09332296997308731
Эпоха 6, Значение функции потерь: 0.08274494111537933
Эпоха 7, Значение функции потерь: 0.1309020221233368
Эпоха 8, Значение функции потерь: 0.10662288218736649
Эпоха 9, Значение функции потерь: 0.6408851146697998
Эпоха 10, Значение функции потерь: 0.07108078896999359
Эпоха 11, Значение функции потерь: 0.06297668814659119
Эпоха 12, Значение функции потерь: 0.0677107647061348
Эпоха 13, Значение функции потерь: 0.09593917429447174
Эпоха 14, Значение функции потерь: 0.08188097923994064
Эпоха 15, Значение функции потерь: 0.09502673149108887
Эпоха 16, Значение функции потерь: 0.05983412265777588
Эпоха 17, Значение функции потерь: 0.05693879723548889
Эпоха 18, Значение функции потерь: 0.07263408601284027
Эпоха 19, Значение функции

In [38]:
y_pred = model(torch.from_numpy(X_test).type(torch.float32))

from sklearn.metrics import classification_report

print(classification_report(y_test, np.argmax(y_pred.detach().numpy(), axis=1)))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     14769
           1       0.00      0.00      0.00      2030

    accuracy                           0.88     16799
   macro avg       0.44      0.50      0.47     16799
weighted avg       0.77      0.88      0.82     16799



c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,